# Import libraries, set options, connect to DB

In [5]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base

passWord = os.environ['my_password']
DATABASE_URI = 'postgres+psycopg2://maxcarey:' + passWord + '@localhost:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

# Read in data as pandas data frame, selecting only certain fields

In [6]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time']

In [7]:
df = pd.read_csv('generated_itineraries.csv', usecols = fields)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Wrange field: destinationIDs

In [8]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

# Remove the 2 cases where the string says null
# Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

# Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)



# Wrangle field: numItenerariesReturned

In [9]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [10]:
#Replace all of the NAs in
df.selectedDestination_name.fillna("", inplace = True)

print("Number of rows before departure date: ")
print(len(df))


Number of rows before departure date: 
25869


# Wrangle Field: departureDate

In [11]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))


# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']


print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
25869
number of rows after removal of anamoulous departureDate cases
25861


# Wrangle Field: distinctID

In [12]:
#It turns out distinc_id correpsonds to a user

# Therefore, create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)


vc = df.primary_key.value_counts()
print(vc[vc > 1])


unique_keys = df.primary_key.unique()
#print(len(unique_keys))

#df.head(n = len(df))

68f3348e-32ff-4756-a6b9-fbf722d5bf76-1460184823                             3
156c0427d8411b-089cb3f5e0f3f-1c114a5c-13c680-156c0427d85c5-1472086519       3
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989343                             3
16017f5f592536-094b655d7b8ce4-7636321b-4a640-16017f5f5932bd-1512207331      3
15ae497624281a-06e6a097b99f59-5e4f2b18-ff000-15ae4976243a16-1489889250      3
c7ca4b64-84f0-46dd-b8e1-00530a09f7d9-1443084797                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989377                             2
6059a95c-9075-49c1-bfce-dc5420759ed3-1469199106                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989392                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989336                             2
1657f9a517fcd8-0eec0990ac17ad-49183707-13c680-1657f9a5180e51-1535419115     2
16693ed1-fd53-4577-8cbe-5ca19ff74b89-1509430935                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989375                 

# Create a subset of the datle with sample method to test geocode and database entry logic

In [13]:
#Out put the entire database
#df.head(len(df))

len(df)

25861

In [14]:
#Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.sample(5)

# Output this random sample
sampleDf.head(len(sampleDf))    

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,primary_key
9837,2017-05-08,15b87f9e0cb2d6-0046bd55f430ac-3c365402-1fa400-...,1,55,,"New Westminster, BC, Canada",1492609782,15b87f9e0cb2d6-0046bd55f430ac-3c365402-1fa400-...
10569,2017-05-08,15beb41c2df55a-0fa4d84c1a1e3f-396e7807-384000-...,1,132,,"522 East 51st Avenue, Vancouver, BC, Canada",1494275893,15beb41c2df55a-0fa4d84c1a1e3f-396e7807-384000-...
12537,2017-06-24,15cdc1b4e2a6f7-0ea5c4e6f38ad5-5393662-cd000-15...,1,157,Tynehead Regional Park,rupert and 26th avenue,1498316478,15cdc1b4e2a6f7-0ea5c4e6f38ad5-5393662-cd000-15...
24764,2018-08-02,164ec51c61f156-0dd9f566ee213b8-4a536b-fa000-16...,1,131,Grouse Grind,"1720 Alberni Street, Vancouver, BC, Canada",1533070543,164ec51c61f156-0dd9f566ee213b8-4a536b-fa000-16...
18607,2018-01-02,160b81f991279-00a5dfb93737c2-44616619-38400-16...,1,112,Baden Powell Lynn Canyon to Grouse,"Waterfront Station West Coast Express, West Co...",1514888822,160b81f991279-00a5dfb93737c2-44616619-38400-16...


## Read in the destination data to allow the possibility to pull the correct names

 


In [15]:
f = open("destinations_mapping_Jul-30-18.csv")

reader = csv.reader(f)


destinations = {}


# The index at the end of the for loop just skips the first row which is the header in the csv file
next(reader)
for row in reader:
    destinations[row[0]] = {'name':row[1]}

print(destinations)

{'168': {'name': 'Mammoth Pass - Crater Meadow Trail'}, '178': {'name': 'Sport Climbing at Horseshoe Slabs'}, '181': {'name': 'Hazel Wolf Wetlands Loop'}, '183': {'name': 'Redwood Grove Loop Trail'}, '184': {'name': 'Birdwatching at Fowlsheugh Reserve'}, '12': {'name': 'Point Defiance Loop (OLD)'}, '112': {'name': 'Baden Powell Lynn Canyon to Grouse'}, '175': {'name': 'Rainbow Falls Trail'}, '174': {'name': 'Walking the Town Loop to Sherwins Vista'}, '20': {'name': 'The Lions Binkert Trail'}, '27': {'name': 'Tumamoc Hill'}, '8': {'name': 'Proximity Alert Test'}, '169': {'name': 'McLeod Lake Spur'}, '165': {'name': 'Siskiyou Mountain (White Rabbit Trail)'}, '22': {'name': 'Lake Blanca'}, '11': {'name': 'Carkeek Park'}, '162': {'name': 'Rouge Park Traverse'}, '172': {'name': 'Walking the Lakes Basin Path at Twin Lakes'}, '16': {'name': 'Golden Gardens to Carkeek Park Beach Walk'}, '111': {'name': 'Baden Powell Deep Cove to Lynn Canyon'}, '278': {'name': 'Tujunga Wash Path'}, '13': {'name

# Loop through the rows in the dataframe, geocode, add entry to database

In [ ]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctKey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to One
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeMapped = mapToPoly(lat, lng)
            
            # TODO:
            # EXPAND THIS MAPPING TO OTHER AREAS
            
            if not selectedDestination_name=row["selectedDestination_name"]:
                
            key = str(destinationID)
            if key in destinations:
            
                # Pull the data out from the dictionary that was created in the cell above
                newName = destinations[str(destinationID)]['name']
        
                # Add the new name to the new row
                rowToModify.selectedDestination_name = newName
            
            # In the case that there is destination that corresponds mark
            else:
                # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
                rowToModify.postalCodeMapped = "DELETED"
                
            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=row["selectedDestination_name"],
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalCode=geocodeInfo['postalCode'],
                                      postalCodeMapped=zipCodeMapped
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=row["selectedDestination_name"],
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")